<img align="center" width="12%" style="padding-right:10px;" src="../Images/Ccom.png">

# Integrated Seabed Mapping Systems <a href="https://teams.microsoft.com/l/channel/19%3af7b302e1f5b84615a207b9e579d62166%40thread.tacv2/Lab%2520D?groupId=81cb9921-c02d-4e3d-b4d7-9ef6985e82b1&tenantId=d6241893-512d-46dc-8d2b-be47e25f5666"><img src="../Images/help.png"  title="Ask questions on Teams" align="right" width="10%" ></a><br><br>  Lab D: Performance Envelope

___

<img align="center" width="80%" style="padding-right:10px;" src="../Images/ccom874_LAB_D_Oblique_Integration_2017.jpg">

This lab is designed for you now that you can navigate the Tx and Rc for each shot time as you did in lab A

We are just going to take one transmission and, for the case of a single receiver beam:
•A –work out where the beam vector was pointing (Azimuth, Depression).
•B –work out where the beam at that TWTT actually hit the seafloor (in a local level, but ship’s head relative coordinate system).
For this you need to do:
•the Cone-Cone intersection Calculation(see separate worked example)

And you need to perform a ray trace. 

In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
from numpy import pi, sin, cos, log, log10, exp, nan
import matplotlib.pyplot as plt
import scipy.ndimage
import gsw
import sys
import os
from pathlib import Path
from datetime import datetime, timezone, timedelta

# The classes and data needed are in the lab A mycode and Data folders respectively

lab_a=Path('../Lab_A/') # Get the path to your Lab_A folder
sys.path.append(str(lab_a.resolve())) # add the Lab_A folder to the list of paths 

# Get the data path
abs_path = os.path.abspath(os.path.curdir)

# import of your own classes

from mycode.Motion import Motion
from mycode.SSP import SSP
from mycode.vessel import Vessel
from mycode.position import Position
from mycode.ping import Ping
from mycode.notebook_to_script import notebook_to_script

# By default represent numbers in numpy arrays with four decimals
np.set_printoptions(formatter={'all':lambda x: ' '+str('%.4f'%x)})

print( )

# Execute the previous step code
# The classes and data needed are in the lab A mycode and Data folders respectively

lab_a=Path('../Lab_A/') # Get the path to your Lab_A folder
sys.path.append(str(lab_a.resolve())) # add the Lab_A folder to the list of paths 

# Get the data path
abs_path = os.path.abspath(os.path.curdir)

# import of your own classes


# By default represent numbers in numpy arrays with four decimals
np.set_printoptions(formatter={'all':lambda x: ' '+str('%.4f'%x)})

# We need a Vessel class object to store the metadata and geometric data descriptive of the vessel

vessel = Vessel()

# The transmit transducer
vessel.lever_arm_trans = np.array([16.26, -1.75,   4.15]).reshape((3, 1))

# The receive transducer
vessel.lever_arm_rec = np.array([14.82, -2.01,   4.17]).reshape((3, 1))

#The Positioning System
vessel.lever_arm_pos = np.array([-5.73, -0.12, -30.00]).reshape((3, 1))

# The Motion Reference Unit
vessel.lever_arm_mru = np.array([0, 0, 0]).reshape((3, 1))
vessel.wl = -2.59

vessel.metadata["name"]="USNS Henson"
vessel.metadata["owned_by"]="United States Navy"
vessel.metadata["operated_by"]="United States Navy"
vessel.metadata["pos_source"]="NavCom (C-Nav)"
vessel.metadata["sonar"]="Kongsberg EM710"
vessel.metadata["mru"]="Applanix POS/MV 320"
vessel.metadata["loa"]=100

beam_n = 391
# We need a Vessel class object to store the metadata and geometric data descriptive of the vessel


# The transmit transducer
vessel.bias_angles_trans = np.array([0.127*pi/180,1.024*pi/180,   359.957*pi/180]).reshape((3, 1))

# The receive transducer
vessel.bias_angles_rec = np.array([0.101*pi/180, 0.894*pi/180,   0.065*pi/180]).reshape((3, 1))
# positions
pos = Position()
pos.read_jhc_file(str(lab_a)+'/Data/Lab_A_GNSS.txt')
# make sure that there is Cartesian representation of the positions
pos.carto_project('utm','ortho')

# Motion data
motions = Motion()
motions.read_jhc_file(str(lab_a)+'/Data/Lab_A_MRU.txt')

# Sound speed data
sound_speed_profile = SSP()
sound_speed_profile.read_jhc_file(str(lab_a)+'/Data/Lab_A_SVP.txt')
ping = Ping()
ping.read('Data/data_ping_4170.txt')
print(type(ping.beam[0]))


Opening GNSS data file:../Lab_A/Data/Lab_A_GNSS.txt
Opening motion data file:../Lab_A/Data/Lab_A_MRU.txt
Opening sound speed profile data file:../Lab_A/Data/Lab_A_SVP.txt
Data/data_ping_4170.txt
Opening sound speed profile data file:Data/data_ping_4170.txt
<class 'numpy.int64'>


___
## D.1 Determine Position and Altitude of the Virtual Array


Now that we have initialized all parameters and read all the data files associated to the ping we may start integrating the data. The first step is to create a virtual array and determine its position. 

<img align="center" width="80%" style="padding-right:10px;" src="../Images/virt_array_pos.jpg">

The virtual array is constructed at the mid point between the Tx location on transmit, and the Rx location on receive. At any single time, the two arrays are actually offset as they cannot be installed at the same place on the hull. Additionally, as these are two separate times, the translation of the entire vessel between those two times should be accounted for.

In real time, as the final navigation solution is not yet available, the convention is to have the stored location of the bottom strike referenced in a coordinate system that is relative to a fixed point in the vessel (the RP not the sonar) as it was at a specific epoch ( almost always the Tx time). That coordinate system is oriented in a local level plane that is aligned along and across the vessel. This is fit for purpose when editing swath data as one normally views the solutions in across track coordinates. That way one can see the shape of successive profiles and perform editing based on their continuity (coherence). Many of the artifacts present in swath data are oriented across the swath (e.g. refraction biases, wobbles), or in the along-track.

For this lab, you will locate the Rx and Tx relative to the RP at that instant using lever arms and instantaneous orientation observations. To account for the along track movement of the RP, the vessel speed and azimuth at that time can be used (and knowing the twtt~ 0.23 seconds). Additionally there will be a small azimuth rotation of the SRF between the two epochs.



### D.1.0 Identify the needed beam

In this step we will find the index of the beam of interest in the `Ping` object, we need this as we will need to find the TWTT associated to that beam in order to calculate the location of the virtual array. It may be that there are beam data missing, so we may not simply use the beam number as an index in the `numpy` arrays contained in the object. Add a method called `get_beam_index()` to the `Ping` class that returns the index of the beam of interest in the array `self.beam` as an integer. If the beam of interest is not in contained in `Ping` object return `None`

In [2]:
# Get the index of the beam if interest and print its value
b_index = ping.get_beam_index(100)
print(b_index)

100


Example for beam 100:
    
    100

___
### D.1.1 Determine the time of Transmit

We need to know the time of transmit `t_Tx` as a `datetime` object. You may find this by combining the `ping.tx_time` with the transmit offset time for the beam of interest i.e., `ping.tx_t_offset_beam[b_index]`

In [3]:
t_Tx = ping.tx_time + ping.tx_t_offset_beam[b_index]
print(t_Tx)

2011-05-08 04:01:06.555400+00:00


Example for beam 100:
    
    2011-05-08 04:01:06.555400+00:00

___
### D.1.2 Determine the time of Reception

We need to know the time of reception `t_Rx` as a `datetime` object. You may find this by combining the `t_tx` with the two way travel time for the beam of interest. 

In [4]:
t_Rx = t_Tx + timedelta( seconds = ping.twtt[b_index])
print(t_Rx)

2011-05-08 04:01:06.712946+00:00


Example for beam 100:
    
    2011-05-08 04:01:06.712946+00:00

___
### D.1.3 Determine the Attitude and Heave at the Time of Transmit

We need to know the attitude and heave at the time of transmit. Add a method to the `Motion` class called `get_motion()` that takes a `datetime` epoch argument and returns the interpolated motion for that epoch as numpy array. The elements should be in the order roll, pitch, yaw, and heave i.e., motions.get_motion(t_Tx)[2] should be the yaw value at `t_Tx`.

The method should look something like this:

        def get_motion(self, time = ...):
            attitude = np.zeros(4)
            times = np.array([e.timestamp() for e in self.times])
            attitude[0] = np.interp(time.timestamp(), times, self.pitch)
            
            ...
            
            return ...

___
### D.1.3.0 Call the `Motion.get_motion()` method

Now use your newly created method to create the `numpy` arrays `att_Tx` and `att_Rx` respectively. These arrays should hold the motion data for the transmit and receive epochs respectively

In [20]:
att_Tx = motions.get_motion(t_Tx)
att_Rx = motions.get_motion(t_Rx)
print(att_Tx)
print(att_Rx)

[ -0.0188  0.0365  3.7564  0.0374]
[ -0.0250  0.0360  3.7612  0.1100]


Example for beam 100:
    
    [ -0.0188  0.0365  3.7564  0.0374]
    [ -0.0250  0.0360  3.7612  0.1100]



___
### D.1.4 Create Rotation Matrices for Attitudes at `t_Tx` and `t_Rx`

To do the required coordinate transformations we need to determine the attitude rotation matrices for the Tx and Rx. The order in which we will rotate is x,y,z (roll, pitch, and yaw). Add method `'get_rot_matrix()` to the `Motion` class that takes as an argument the time and returns the compound rotation matrix - note that you may call your just generated  `Motion.get_motion()` method from this method for this purpose, i.e.,

        def get_rot_matrix(self, time = ...):
            att = self.get_motion(time)
            
            ...
            
            return ...
            
Note that you may use Lab A Step 3.2.8 (code in the `Integration` class) as inspiration for the creation of your rotation matrices, the difference being that you are returning a single matrix for the epoch of interest

Now use your newly created method to create the `numpy` arrays `R_Tx` and `R_Rx` respectively. These arrays should hold the rotation matrices for the transmit and receive epochs respectively

In [25]:
R_Tx = motions.get_rot_matrix(t_Tx)
R_Rx = motions.get_rot_matrix(t_Rx)
print(R_Tx)
print(R_Rx)

[[ 0.9996  0.0191  -0.0184]
 [ -0.0188  0.9996  0.0191]
 [ 0.0188  -0.0188  0.9996]]
[[ 0.9994  0.0256  -0.0243]
 [ -0.0250  0.9994  0.0256]
 [ 0.0250  -0.0250  0.9994]]


___
### D.1.5 Determine the Position at the Time of Transmit and Reception

We need to know the Position at the time of transmit and reception. Add a method to the `Position` class called `get_position()` that takes a `datetime` epoch argument and returns the interpolated position for that epoch as a numpy array. The elements should be in the order E, N, height as defined by the `Position.carto_project()` method.

Now use your newly created method to create the `numpy` arrays `pos_Tx` and `pos_Rx` respectively. These arrays should hold the position data for the transmit and receive epochs respectively.

For ins


In [23]:
pos_Tx = pos.get_position(t_Tx)
pos_Rx = pos.get_position(t_Rx)
print(pos_Tx)
print(pos_Rx)

[ 396628.0735  1764532.9556  28.5177]
[ 396627.8849  1764532.2738  28.4402]


___
### D.1.6 Calculate the Georeferenced Reference Position at the Transmit (`t_TX`) and Receive (`t_Rx`) Epochs

The positions retrieved in D.1.5 are the georeferenced positions of the positioning system. We want to establish where the reference point RP of the vessel reference frame is located in georeferenced space. Thus we need to transform the lever arms from the vessel reference frame to the georeferenced frame, we saw an example in Lab A of how to achieve this by using the lever arms of the positioning system in the ship's reference frame. 

Note that we have the choice here to use geodesic distances along the ellipsoid, which minimizes distortion, or distances in a projected Cartesian coordinate system. Considering the relatively small distances, and the achievable positioning uncertainties in the data, we chose to do this in a projected coordinate system as the induced distortions will be small and the math will be much simpler. 

#### D.1.6.1 Calculate the Positioning System Lever Arm at the Transmit (`t_TX`) and Receive (`t_Rx`) Epochs

Add a method to the `Motion` class called `geo_reference_la()` that takes a `datetime` epoch and a lever arm argument consistent with the lever arms as defined in the `Vessel` class (e.g., `vessel.lever_arm_pos`) and returns the georeferenced lever arm for that epoch as a numpy array. The elements should represent the distance in the East, North and Up directions in meters. You may use Lab A step 3.2.9 as inspiration. Similar to before you may choose to call the `get_rot_matrix` method from this method.

In [29]:
la_Tx = motions.geo_reference_la(t_Tx, lever_arm_trans)
la_Rx = motions.geo_reference_la(t_Rx, ...)
print(rp_Tx)
print(rp_Tx)

None
None


___
#### D.1.6.2 Determine the reference position at the time of transmit

Now that we know the lever arms you may determine the position of the reference point at transmit and receive times. Add a method called `pos_to_rp` to the Position class to which you pass the lever arms you just created and which returns the georeferenced location of the reference position on the vessel

In [29]:
rp_Tx = motions.pos_to_rp(t_Tx, ...)
rp_Rx = motions.pos_to_rp(t_Rx, ...)
print(rp_Tx)
print(rp_Tx)

None
None


In [ ]:
D_1_1 = 'The virtual array was located using the following assumptions: ...'